## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-24-22-52-15 +0000,nypost,"How Trump plans to fund $2,000 tariff ‘dividen...",https://nypost.com/2025/11/24/business/how-tru...
1,2025-11-24-22-51-41 +0000,nypost,"Teen driver intentionally caused 100 mph, 12-c...",https://nypost.com/2025/11/24/us-news/teen-dri...
2,2025-11-24-22-51-29 +0000,nyt,Xi Presses Trump on Taiwan as They Agree to Me...,https://www.nytimes.com/2025/11/24/business/ec...
3,2025-11-24-22-49-00 +0000,wsj,Opinion | A Tale of Two Failing Cease-Fires,https://www.wsj.com/opinion/a-tale-of-two-fail...
4,2025-11-24-22-49-00 +0000,wsj,Alimentation Couche-Tard Second-Quarter Profit...,https://www.wsj.com/business/earnings/alimenta...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2383/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,37
61,ukraine,21
53,new,17
62,peace,16
63,plan,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
88,2025-11-24-20-17-58 +0000,nypost,"US, Ukraine agree on new 19-point peace plan t...",https://nypost.com/2025/11/24/world-news/us-uk...,124
247,2025-11-24-07-31-06 +0000,nyt,"Ukraine, U.S. Meet About Trump’s Peace Plan to...",https://www.nytimes.com/2025/11/23/world/europ...,114
246,2025-11-24-07-32-21 +0000,nyt,What Is Trump’s 28-Point Plan to End Russia’s ...,https://www.nytimes.com/2025/11/22/world/europ...,99
7,2025-11-24-22-43-00 +0000,wsj,Ukraine Peace-Plan Negotiators Leave Thorniest...,https://www.wsj.com/world/europe/big-gaps-stil...,97
279,2025-11-23-23-12-21 +0000,nypost,Leaked EU peace deal calls for Ukraine to keep...,https://nypost.com/2025/11/23/world-news/leake...,97


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
88,124,2025-11-24-20-17-58 +0000,nypost,"US, Ukraine agree on new 19-point peace plan t...",https://nypost.com/2025/11/24/world-news/us-uk...
127,52,2025-11-24-17-49-48 +0000,nypost,Judge dismisses cases against ex-FBI Director ...,https://nypost.com/2025/11/24/us-news/judge-di...
98,50,2025-11-24-19-37-47 +0000,nypost,Elderly man fatally stabbed in his NYC apartme...,https://nypost.com/2025/11/24/us-news/elderly-...
114,49,2025-11-24-18-33-00 +0000,wsj,President Trump’s advisers are discussing a pr...,https://www.wsj.com/politics/policy/trump-advi...
13,34,2025-11-24-22-22-05 +0000,wapo,"In Gulf of America case, AP renews legal fight...",https://www.washingtonpost.com/business/2025/1...
82,31,2025-11-24-20-41-07 +0000,nypost,‘Godfather’ actor wildly claims Pope John Paul...,https://nypost.com/2025/11/24/world-news/godfa...
254,30,2025-11-24-04-44-04 +0000,bbc,Australia is banning social media for kids und...,https://www.bbc.com/news/articles/cwyp9d3ddqyo...
27,29,2025-11-24-21-59-32 +0000,nypost,Trump accepts Xi’s invite to Beijing and calls...,https://nypost.com/2025/11/24/us-news/trump-ac...
125,28,2025-11-24-18-03-20 +0000,nyt,"Jamil Abdullah Al-Amin, Black Power Activist K...",https://www.nytimes.com/2025/11/23/us/h-rap-br...
124,28,2025-11-24-18-04-19 +0000,nypost,Putin believes Russia-friendly US cease-fire p...,https://nypost.com/2025/11/24/world-news/russi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
